In [1]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

In [2]:
df = pd.read_excel('task2_stock_data.xlsx')

In [3]:
def calculateBeta (asofdate, window, frequency):
    #calculate start date based on file
    firstdate=df.iloc[0,0]
    
    #set period according to window entered
    if window=="1y":
        period=-1
    elif window=="2y":
        period=-2
    else:
        print("Incorrect window entered. Options are: '1y' or '2y'.")
        return
    
    try:
        windowstart = pd.to_datetime(asofdate) + pd.DateOffset(years=period)
    except:
        print("Incorrect asofdate entered. Please use the following format: 'YYYY-MM-DD'")
        return
    
    if firstdate > pd.to_datetime(asofdate) + pd.DateOffset(years=period):
        print (f"WARNING: The time between the first date in the file <{firstdate.strftime('%Y-%m-%d')}> and the given asofdate <{asofdate}> is less than the given window <{window}>. The calculation is therefore performed on a shorter window of returns.")
    
    #trim the dataframe by asofdate and period
    #sort so that the return calculation includes the latest returns
    window_df = df.loc[(df['date'] < asofdate) & (df['date'] >= windowstart)].sort_values(by='date', ascending=False) 
    
    #set freq based on frequency entered
    if frequency=="daily":
        freq=1
    elif frequency=="weekly":
        freq=5
    elif frequency=="bi-weekly":
        freq=5*2
    elif frequency=="monthly":
        #assumption. 20.97 was the average days in a month in the given file
        freq=21
    elif frequency=="quarterly":
        #assumption. multiplied monthly by 4
        freq=21*4
    else:
        print("Incorrect frequency entered. Options are: 'daily','bi-weekly','monthly', or 'quarterly'.")
        return
        
    #trim the dataframe by frequency
    freq_window_df = window_df.iloc[::freq, :]
    
    #calculate returns, winsorize, and store in a dictionary
    freq_window_df.set_index("date", inplace=True)
    returns={}
    for column in freq_window_df:
        data = freq_window_df[column]
        log_returns = np.log(data/data.shift())
        log_returns_winsorized = winsorize(log_returns,limits=[0.05, 0.05])
        returns[column]=log_returns_winsorized[1:]
    
    #calculate covariances and store in a dictionary
    covariances={}
    for key in returns:
        covariances[key] = np.cov(returns[key],returns["SPY US Equity"],bias=True)[1][0]
    
    #calculate market variance
    market_variance=np.var(returns["SPY US Equity"])
    
    #calculate betas store in a dictionary. Rounded to 10 due to lost precision issues
    betas = {}
    for key in covariances:
        betas[key]=round((covariances[key]/market_variance),10)
    
    #return the result
    return betas
    

In [4]:
calculateBeta("2021-10-31","1y","daily")

{'APTMU US Equity': nan,
 'AAC US Equity': nan,
 'CWEN US Equity': 0.8989808641,
 'BHC US Equity': 1.0212363,
 'FACA US Equity': nan,
 'AFT US Equity': 0.1748769343,
 'ARBG US Equity': nan,
 'AAC/WS US Equity': nan,
 'BKKT US Equity': nan,
 'CWEN/A US Equity': 0.9708069857,
 'AGAC US Equity': nan,
 'ARBGU US Equity': nan,
 'FACA/WS US Equity': nan,
 'ACAQ/U US Equity': nan,
 'ARGUU US Equity': nan,
 'CYDY US Equity': 0.3960686469,
 'FACT US Equity': nan,
 'AGAC/WS US Equity': nan,
 'BKLN US Equity': 0.1119000666,
 'ACDI/U US Equity': nan,
 'ASZ US Equity': nan,
 'FLAG/U US Equity': nan,
 'BWCAU US Equity': nan,
 'AGGRU US Equity': nan,
 'ADALU US Equity': nan,
 'CYRX US Equity': 1.3634391243,
 'AHLD US Equity': nan,
 'FLYA/U US Equity': nan,
 'DCRN US Equity': nan,
 'BYND US Equity': 0.7323806122,
 'AEAEU US Equity': nan,
 'ATUS US Equity': 0.7624803832,
 'AMCIU US Equity': nan,
 'FMIVU US Equity': nan,
 'AFACU US Equity': nan,
 'AVAN US Equity': nan,
 'DDD US Equity': 2.3167928838,
 '

In [5]:
calculateBeta("2021-10-31","1y","weekly")

{'APTMU US Equity': nan,
 'AAC US Equity': nan,
 'CWEN US Equity': 1.7677381811,
 'BHC US Equity': 1.4023917239,
 'FACA US Equity': nan,
 'AFT US Equity': 0.262832467,
 'ARBG US Equity': nan,
 'AAC/WS US Equity': nan,
 'BKKT US Equity': nan,
 'CWEN/A US Equity': 1.7688540814,
 'AGAC US Equity': nan,
 'ARBGU US Equity': nan,
 'FACA/WS US Equity': nan,
 'ACAQ/U US Equity': nan,
 'ARGUU US Equity': nan,
 'CYDY US Equity': 1.2724733897,
 'FACT US Equity': nan,
 'AGAC/WS US Equity': nan,
 'BKLN US Equity': 0.1126273417,
 'ACDI/U US Equity': nan,
 'ASZ US Equity': nan,
 'FLAG/U US Equity': nan,
 'BWCAU US Equity': nan,
 'AGGRU US Equity': nan,
 'ADALU US Equity': nan,
 'CYRX US Equity': 4.1562752877,
 'AHLD US Equity': nan,
 'FLYA/U US Equity': nan,
 'DCRN US Equity': nan,
 'BYND US Equity': 0.6298104042,
 'AEAEU US Equity': nan,
 'ATUS US Equity': 0.9176213371,
 'AMCIU US Equity': nan,
 'FMIVU US Equity': nan,
 'AFACU US Equity': nan,
 'AVAN US Equity': nan,
 'DDD US Equity': 4.8782393385,


In [6]:
# # checking to see if the result is identical to the linear regression method.
# # I chose to calculate covariance and variance to:
# # (1) better understand the calculation in-depth, and
# # (2) my guess is that linear regression is slower and/or more memory intensive

# from sklearn.linear_model import LinearRegression

# X = returns['SPY US Equity'].reshape(-1, 1)
# Y = returns['CWEN US Equity'].reshape(-1, 1)
 
# lin_regr = LinearRegression()
# lin_regr.fit(X, Y)
 
# lin_regr.coef_[0, 0]
